# Objetivo: ler e analisar as séries históricas baixadas <EM CONSTRUÇÃO>

Referência: https://github.com/Schiessl/AgoraVAI/blob/main/Importa_S%C3%A9rie_Hist%C3%B3rica_B3_Todas_as_a%C3%A7%C3%B5es_e_op%C3%A7%C3%B5es_v1_0_0.ipynb

### Importa bibliotecas

In [1]:
import pandas as pd
from datetime import datetime
import os
import zipfile

### Layout do arquivo
Arquivo disponível em: https://www.b3.com.br/data/files/33/67/B9/50/D84057102C784E47AC094EA8/SeriesHistoricas_Layout.pdf

In [2]:
layout = [
    ("tipo_registro", 2, 'str'),
    ("data_pregao", 8, 'datetime'),
    ("cod_bdi", 2, 'str'),
    ("cod_negociacao", 12, 'str'),
    ("tipo_mercado", 3, 'str'),
    ("nome_empresa", 12, 'str'),
    ("especificacao_papel", 10, 'str'),
    ("prazo_dias_merc_termo", 3, 'str'),
    ("moeda_referencia", 4, 'str'),
    ("preco_abertura", 13, 'float'),
    ("preco_maximo", 13, 'float'),
    ("preco_minimo", 13, 'float'),
    ("preco_medio", 13, 'float'),
    ("preco_ultimo_negocio", 13, 'float'),
    ("preco_melhor_oferta_compra", 13, 'float'),
    ("preco_melhor_oferta_venda", 13, 'float'),
    ("numero_negocios", 5, 'int'),
    ("quantidade_papeis_negociados", 18, 'int'),
    ("volume_total_negociado", 18, 'float'),
    ("preco_exercicio", 13, 'float'),
    ("indicador_correcao_precos", 1, 'str'),
    ("data_vencimento", 8, 'datetime'),
    ("fator_cotacao", 7, 'int'),
    ("preco_exercicio_pontos", 13, 'float'),
    ("codigo_isin", 12, 'str'),
    ("num_distribuicao_papel", 3, 'str')
]

Alterei a função original para permitir a leitura do arquivo diretamente do ZIP

In [3]:
def importar_arquivo_b3(caminho_arquivo, nrows=None):
    """
    Função para importar dados de um arquivo no formato B3 (BM&FBOVESPA) e verificar a consistência.

    Parâmetros:
    - caminho_arquivo (str): O caminho do arquivo a ser importado.
    - nrows (int or None): Número de linhas a serem lidas do arquivo. Se None, lê todas as linhas.

    Retorna:
    - pd.DataFrame: Um DataFrame contendo os dados importados.

    A função lê o arquivo, realiza a importação para um DataFrame e verifica a consistência comparando a quantidade
    de registros importados com a quantidade informada no trailer.
    """
    inicio = datetime.now()
    print('Inicio da leitura:', inicio)

    # Definir os tipos de dados para cada coluna
    dtype_mapping = {
        field[0]: (
            'datetime64[ns]' if field[2] == 'datetime' else
            (float if field[2] == 'float' else
            (int if field[2] == 'int' else 'str'))
        ) for field in layout
    }

    largura_campo = [field[1] for field in layout]

    # Ler o arquivo diretamente com Pandas, lendo apenas as primeiras linhas se nrows for especificado
    df = pd.read_fwf(
        caminho_arquivo,
        compression='zip',
        widths=largura_campo,
        dtype=dtype_mapping,
        nrows=nrows,  # Número de linhas a serem lidas
        skipheader=1, # pula o header
        skipfooter=1 if nrows is None else 0 # pula o trailer quando lê arquivo completo
    )

    # Renomear as colunas
    df.columns = [field[0] for field in layout]

    # Converter a colunas com datas para datetime64[ns]
    df['data_pregao'] = pd.to_datetime(df['data_pregao'], format='%Y%m%d', errors='coerce')
    df['data_vencimento'] = pd.to_datetime(df['data_vencimento'], format='%Y%m%d', errors='coerce')
    # Arredondar campos para duas casas decimais
    decimal_fields = [field[0] for field in layout if field[2] == 'float']
    df[decimal_fields] = df[decimal_fields] / 100
    # Formatar o campo 'preco_exercicio_pontos' com 6 casas decimais
    df['preco_exercicio_pontos'] = df['preco_exercicio_pontos'] / 1e6
    # Aplicar formatação específica apenas para o campo volume_total_negociado
    df['volume_total_negociado'] = df['volume_total_negociado'].apply(lambda x: '{:.2f}'.format(x))

    # Exibir a quantidade de linhas importadas
    # print('Quantidade de linhas importadas:', len(df))

    if nrows is None:
        # Se lendo todas as linhas, verificar o trailer
        # Exibir a quantidade lida no trailer (posições 32 a 42)
        with zipfile.ZipFile(caminho_arquivo, 'r') as zip_ref:
            with zip_ref.open(os.path.basename(caminho_arquivo).replace('.ZIP', '.TXT'), 'r') as file:
                lines = file.readlines()
                trailer = lines[-1]
                quantidade_lida_no_trailer = int(trailer[32:42])-2  # exclui header e trailer

        if len(df) == quantidade_lida_no_trailer:
            print(f'\n#### Arquivo importado com sucesso! - {len(df):,.0f} registros em {(datetime.now() - inicio)} ####')
        else:
            print(f'\n---- Arquivo não foi importado corretamente! ----')
            print(f'---- Foram importadas {len(df):,.0f} linhas, mas o trailer indica {quantidade_lida_no_trailer} linhas. ----')
    else:
            print(f'\n---- Foram importadas {len(df):,.0f} linhas. ----')

    print('\nFim da leitura:', datetime.now())
    return df

In [4]:
# Exemplo da leitura de um arquivo
caminho_do_arquivo = 'C:/repos/b3/series_historicas/dados/avaliar/COTAHIST_A2020.ZIP'

df = importar_arquivo_b3(caminho_do_arquivo)

Inicio da leitura: 2024-04-25 09:35:50.542493

#### Arquivo importado com sucesso! - 1,251,646 registros em 0:00:28.807120 ####

Fim da leitura: 2024-04-25 09:36:19.350421


In [5]:
# Mostra aleatóriamente alguns registros
df.sample(5)

,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,nome_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,preco_abertura,...,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,indicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel
562463,1,2020-05-12,82,BBDCR500,80,BBDCE /EJ,PN N1,0.0,R$,1.64,...,10,6630,11746.40,18.66,0,2020-06-15,1,0.0,BRBBDCACNPR8,708
43293,1,2020-02-28,8,RNEW11,10,RENOVA,UNT N2,NaN,R$,11.63,...,22,3100,36957.00,0.00,0,NaT,1,0.0,BRRNEWCDAM15,106
1147281,1,2020-11-30,82,ITUBX266,80,ITUBE,PN N1,0.0,R$,0.33,...,22,24700,8156.00,26.54,0,2020-12-21,1,0.0,BRITUBACNPR1,268
59804,1,2020-03-20,96,IRBR3F,20,IRBBRASIL RE,ON NM,NaN,R$,8.05,...,6081,185976,1526108.41,0.00,0,NaT,1,0.0,BRIRBRACNOR4,106
645594,1,2020-06-10,82,ELETS310,80,ELETE FM,PNB N1,0.0,R$,2.03,...,13,76900,166722.00,31.00,0,2020-07-20,1,0.0,BRELETACNPB7,71
